In [1]:
import csv                       #use library csv to open and read csv data
import numpy as np 
import math
import datetime
import random
path='/home/mz/Desktop/winequality-red.csv'

with open(path,'r') as winedata:
    reader = csv.reader(winedata, delimiter=';') #maybe csv from mac? Set the delimiter to ';'
    rows= [row for row in reader]
    
#print (rows[0])
data=np.array(rows)               #change rows' datatype into array
dataset=np.delete(data,0,axis = 0) #delete the name row

#wdataset will be used for K-fold validation
wdataset = dataset.astype(np.float)
for n in range(0,wdataset.shape[0]): 
    #classify the quality. Positiv:1  Negativ:2
    if wdataset[n,11]>5: wdataset[n,11]=1
    else: wdataset [n,11]=0
print(wdataset)

datash = wdataset
np.random.shuffle(datash)
data0=datash




k=5
n0 = data0.shape[0]
X_test = data0[0:int(n0/k), 0:-1 ]
y_test = data0[0:int(n0/k), -1 ]
data2 = np.delete(data0, range(0,int(n0/k)),axis = 0)#delete testset from data
n1 = data2.shape[0]#number of rows in data2 for training and validation
X_all = data2[:,0:-1]
y_all = data2[:,-1]
X_valid = [[]for i in range(k)]
X_train = [[]for i in range(k)]
y_valid = [[]for i in range(k)]
y_train = [[]for i in range(k)]
trainset = [[]for i in range(k)]
for i in range(0,k):
    X_valid[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), 0:-1])
    y_valid[i] = np.array(data2[int((i*n1)/k):int((i+1)*n1/k), -1])
    a=np.arange(int((i*n1)/k),int((i+1)*n1/k))
    trainset[i] = np.delete(data2, a , axis=0)
    X_train[i] = trainset[i][:,0:-1]
    y_train[i] = trainset[i][:,-1]
    print('valid i=', i, X_train[i].shape)
    print('valid i=', i, y_train[i].shape)

[[ 7.4    0.7    0.    ...  0.56   9.4    0.   ]
 [ 7.8    0.88   0.    ...  0.68   9.8    0.   ]
 [ 7.8    0.76   0.04  ...  0.65   9.8    0.   ]
 ...
 [ 6.3    0.51   0.13  ...  0.75  11.     1.   ]
 [ 5.9    0.645  0.12  ...  0.71  10.2    0.   ]
 [ 6.     0.31   0.47  ...  0.66  11.     1.   ]]
valid i= 0 (1024, 11)
valid i= 0 (1024,)
valid i= 1 (1024, 11)
valid i= 1 (1024,)
valid i= 2 (1024, 11)
valid i= 2 (1024,)
valid i= 3 (1024, 11)
valid i= 3 (1024,)
valid i= 4 (1024, 11)
valid i= 4 (1024,)


In [3]:
def sigmoid(gamma):#the sigmoid function we used in gradient descenting and prediction. To avoid overflowing, input of exp is always a negative number
    if gamma < 0:
        return 1 - 1/(1 + math.exp(gamma))
    else:
        return 1/(1 + math.exp(-gamma))
    

'''
def add(data,w):
            m = data.shape[1]
            summe = np.zeros(m-1)
            for i in range(0,data.shape[0]):
                a = data[i,0:(data.shape[1]-1)]*((data[i,(data.shape[1]-1)])-sigma(w, data[i,0:(data.shape[1]-1)]))
                summe = summe + a         
            return summe    
    
def gd(w,lr,data):
            neww = w + lr*add(data,w)
            return neww
'''
#lr learning rate; n iteration times; data data used for training; y_train:label of training data
def findw(lr,n,data,y_train):
    traindata = data
    m = traindata.shape[1]#column number of input data(features)
    #w = [0,0,0,0,0,0,0,0,0,0,0]#除去最后一列的quality
    #w = [1,1,1,1,1,1,1,1,1,1,1]
    w = np.ones(m)#define w
    #right = [0,0,0,0,0,0,0,0,0,0,0]
    #vector
    right = np.zeros(m)
    for i in range(1,n):#n times iteration
        
        for j in range(0,traindata.shape[0]):
            right = right + traindata[j,]*(y_train[j]-sigmoid(np.dot(w,traindata[j,])))
            
        for l in range(0,right.shape[0]):
            right[l]=right[l]*lr
        #print(right)
        w=w+right
        #print(w)
    return w


#data.TEST DATA; w:w from findw function
def predict(data,w):#use w which calculated by findw function to predict the classification of input data
        n = data.shape[0]
        result = np.zeros(n)
        for i in range(0,n):
            prob = sigmoid(np.dot(w,data[i,]))
            #prob = (1/math.sqrt(2*pi*sigma(w,data[i,0:n-1]))) * math.exp(-(1-np.transpose(w)*data[i,0:n-1]/2(sigma(w,data[i,0:n-1])*sigma(w,data[i,0:n-1]))))
            if prob>=0.5: result[i]=1
        return result
#pred: result from predict function; y: real value(label of training data)            
def accuracy(pred, y):#pred is result from fct predict
    real = y
    
    if pred.shape != real.shape:
        print('Error!!!')
        
    acc=0
    for i in range(pred.shape[0]):
        if pred[i]==real[i]:
            acc=acc+1
    Accuracy=acc/pred.shape[0]   
    #print('Accuracy is ',Accuracy)
    return Accuracy

#result: result from predict function; y: real value
def conmat(result,y):#return confusion matrix for prediction
    rv = y
    print(result.shape)
    print(rv.shape)
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(result.shape[0]):
        if ((result[i]==1)&(rv[i]==1)): TP = TP+1
        if ((result[i]==0)&(rv[i]==0)): TN = TN+1
        if ((result[i]==0)&(rv[i]==1)): FN = FN+1
        if ((result[i]==1)&(rv[i]==0)): FP = FP+1
    metrix = np.array([[TP,FP], [FN,TN]])
    return metrix


def fit():#tuning training and testing; output all informations we need
    start = datetime.datetime.now()
    a_n = 7#number of groups of alpha
    iter_n= 5#number of n
    numwewant = 40#number of randomly chosed pairs
    #give alpha values in 1dB step
    alpha = np.zeros(9*a_n)
    for i in range(9):
        alpha[i]= 1e-7*(i+1)
        alpha[i+9]= 1e-6*(i+1)
        alpha[i+18]= 1e-5*(i+1)
        alpha[i+27]= 1e-4*(i+1)
        alpha[i+36]= 1e-3*(i+1)
        alpha[i+45]= 1e-2*(i+1)
        alpha[i+54]= 1e-1*(i+1)
        #alpha[i+63]= (i+1)
        #alpha[i+72]= (i+1)*10
        #alpha[i+81]= (i+1)*100
    
    #n_iteration = [500, 1000, 1500]
    n_iteration = [500, 1000, 1500, 2500, 3000]
    
    pairs = [[]for i in range(a_n*iter_n*9)]
    n = 0
    
    #offer all possibilities for pairs of alpha and iteration
    for i in range(9*a_n):
        j = 0
        while j<5:
            pairs[n] = [alpha[i], n_iteration[j]]
            n = n+1
            j = j+1
             
    print(pairs)  
    
    print('\n')
    #print(pairs)
    
    #here we chose numwewant from 9*a_n*5
    para_pos = np.array(random.sample(range(0,a_n*iter_n*9), numwewant))#generate used pairs's position
    #result = np.zeros([80,256])
    result = [[]for i in range(numwewant)]#save the predict results
    print(para_pos)
    A = np.zeros(numwewant)#the avg accuracy vector for 80 pairs
    Accu = np.zeros([numwewant,5])#Accuracy for each validation
    #ln=0
    
    for i in range(numwewant):
        print('\n')
        pos = para_pos[i]
        print('pos',pos)
        para = pairs[pos]
        print('para',para)
        for j in range(5):
            #a = para[0]
            #b = para[1]
            #print(a)
            #print(b)
            lr = para[0]
            n = para[1]
            w_model = findw(lr,n,X_train[j],y_train[j])#find w according to X_train and y_train
            print('fold',j,'of model',i,'finished',w_model)
            result[i] = predict(X_valid[j],w_model)#using X_valid as input to produce prediction output
            #result = predict(X_valid[j],w_model)
            #result[ln] = predict(X_valid[j],w_model)
            #ln = ln+1
            Accu[i,j]= accuracy(result[i], y_valid[j])#compare result(output prediction) and real values y_valid
            print('Accuracy is',Accu[i,j])
            if j ==4: A[i] = np.sum(Accu[i,:])/5
            if j ==4: print('\033[1;35m Average accuracy is \033[0m',A[i])          
            print('\n')
            
            
  


        
    a = np.max(A)
    pos_max = np.where(A==a)
    location = pos_max[0][0]#location is a number which used to locate which random number in random vector
    print('used position',location)
    bestalpha = pairs[para_pos[location]][0]#use location to find best alpha and n
    bestn = pairs[para_pos[location]][1]
    print('avg A is',A)
    print('max',np.max(A))
    print('max_pos',pos_max)
    print('location',location)
    print(pairs[para_pos[location]])
    print('best para are',bestalpha,bestn)
    
    w_final = findw(bestalpha, bestn, X_all, y_all)
    final_result = predict(X_test,w_final)#using testset to verify and produce prediction results
    finalaccu = accuracy(final_result,y_test)#calculate the accuracy according to label y_test and prediction results
    finalco = conmat(final_result,y_test)
    print('The final accuracy is', finalaccu)
    print('The confusion matrix of prediction is\n', finalco)
    end = datetime.datetime.now()
    print ('used time is',end-start)
    return w_final





In [3]:
fit()#1st

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 3 of model 2 finished [ 0.62171787  0.97275332  0.98961432  0.88774628  0.99568929  0.42153071
 -0.51193035  0.95472675  0.84955972  0.97258674  0.55322451]
Accuracy is 0.61328125


fold 4 of model 2 finished [ 0.64023056  0.97415159  0.99027629  0.89494254  0.99606191  0.42733043
 -0.52175264  0.95704902  0.8573691   0.97434893  0.57441848]
Accuracy is 0.60546875
 Average accuracy is  0.62421875




pos 12
para [3e-07, 1500]
fold 0 of model 3 finished [ 0.23195797  0.94233928  0.98191795  0.78288707  0.99146037  0.20765766
 -0.2523253   0.90795864  0.69335693  0.94780358  0.09475035]
Accuracy is 0.59765625


fold 1 of model 3 finished [ 0.19005171  0.94087048  0.9806679   0.76921417  0.99146166  0.17453335
 -0.22560143  0.90545567  0.68510749  0.94728895  0.0715842 ]
Accuracy is 0.66796875


fold 2 of model 3 finished [ 0.23002574  0.94162485  0.98269969  0.79161614  0.99168743  0.15454536
 -0.22814551  0.90748548  0.69046552  0.94753964  0.09270538]
Accuracy is 0.609375


fold 3

fold 1 of model 10 finished [-150.33394913 -635.77163061  292.53489466  -36.96150175  -42.08479631
  321.27819828  -73.47829532 -216.35934527 -810.48306507  339.46435386
  589.43122779]
Accuracy is 0.55859375


fold 2 of model 10 finished [  40.79643796 -651.52358136  288.74496837   29.59107224  -42.69469744
  309.20083477  -73.10605807 -235.11926356 -868.98420878  289.45477338
  488.77164254]
Accuracy is 0.5078125


fold 3 of model 10 finished [-162.10796653 -618.35214804  264.82873583 -106.04894132  -72.51459576
  248.8067881   -67.69430991 -242.4271424  -872.36971814  230.35493103
  685.4522335 ]
Accuracy is 0.62109375


fold 4 of model 10 finished [ -98.47367273 -555.71069533  240.94235082  -38.64995077  -46.17045272
  259.35161278  -69.09262266 -198.37474837 -696.29790154  262.03043299
  577.00757832]
Accuracy is 0.5703125
 Average accuracy is  0.56484375




pos 270
para [0.1, 500]
fold 0 of model 11 finished [  616.71619413 -1790.87080166   865.05410254  -115.59433112
  -146.223

fold 0 of model 18 finished [ -3332.25508752 -20201.46850153   9003.44811916  -1003.33546865
  -1797.74851772   6072.42696678  -8789.61054966  -7997.31804833
 -28722.09147778   8669.99479297  15734.06195942]
Accuracy is 0.4921875


fold 1 of model 18 finished [ -7509.17337358 -20868.79940019   9739.172022     -913.30366051
  -1420.66964211   7799.72799602  -2033.55254177  -7505.15529589
 -27827.59846193  10088.4633139   21634.7652508 ]
Accuracy is 0.57421875


fold 2 of model 18 finished [ -6541.77496391 -20951.55141927   9688.71849458   2427.03909421
  -1432.36650256   2209.90763297  -5378.71494555  -8377.6104197
 -30773.12604746   7899.95181933  16717.83358792]
Accuracy is 0.53515625


fold 3 of model 18 finished [ -6531.27938397 -20590.58577978   9068.03365355  -2031.79917935
  -2260.57702737   7854.81031202  -1186.50114582  -8422.7767544
 -29964.97545172   6045.9157099   24010.67689404]
Accuracy is 0.5703125


fold 4 of model 18 finished [ -4784.52543878 -18552.05134401   8082.1234

fold 0 of model 26 finished [  4.34450079 -37.22906052  18.45635361  -3.06582366  -2.28033836
  31.80220108  -8.20876078 -13.13634184 -50.19882041  18.67110044
  35.9366917 ]
Accuracy is 0.5546875


fold 1 of model 26 finished [ -9.87165264 -38.14569228  19.19794308  -2.11513084  -1.62694297
  32.66238434  -7.85789398 -12.19409725 -48.01276384  21.74750857
  44.12855142]
Accuracy is 0.5625


fold 2 of model 26 finished [  1.52792066 -39.37009992  19.11779602   3.77873418  -1.63176314
  25.88800347 -22.04529136 -13.67132059 -53.06110858  18.63329736
  32.92704446]
Accuracy is 0.6015625


fold 3 of model 26 finished [-10.33973531 -36.88621597  17.39697686  -9.09628715  -3.40182054
  26.73990462  -7.40571781 -13.51422689 -50.89537982  15.11785477
  50.82018092]
Accuracy is 0.62109375


fold 4 of model 26 finished [ -5.44851504 -32.67758717  15.49909931  -2.44718216  -1.87602042
  28.87100171  -8.25289134 -10.8428893  -40.03614709  16.81032341
  42.72409022]
Accuracy is 0.58984375
 Average

fold 0 of model 34 finished [ 0.56653624  0.96810943  0.98886448  0.87399388  0.99513609  0.39261188
 -0.46453811  0.94763624  0.8256685   0.96925488  0.4813468 ]
Accuracy is 0.609375


fold 1 of model 34 finished [ 0.54905042  0.96755167  0.98832443  0.86814767  0.99515796  0.38144972
 -0.45018209  0.94671363  0.82268744  0.96905524  0.4731911 ]
Accuracy is 0.671875


fold 2 of model 34 finished [ 0.56963164  0.9678997   0.98941459  0.87827554  0.99531333  0.36977699
 -0.45118925  0.94776743  0.82549818  0.96940103  0.48337972]
Accuracy is 0.6171875


fold 3 of model 34 finished [ 0.55096216  0.96752706  0.9880059   0.86897519  0.99490964  0.38427155
 -0.46825182  0.94658839  0.82254414  0.96803411  0.47491899]
Accuracy is 0.61328125


fold 4 of model 34 finished [ 0.57579413  0.96928221  0.98890747  0.87823966  0.99539462  0.39208329
 -0.48126691  0.94963083  0.83272902  0.97033559  0.50216778]
Accuracy is 0.60546875
 Average accuracy is  0.6234375




pos 255
para [0.07, 500]
fold 0

array([ 0.57946147,  0.96936683,  0.98905814,  0.87803884,  0.99536207,
        0.39382107, -0.4736366 ,  0.94965473,  0.83244912,  0.97028899,
        0.50231899])

In [6]:
fit()#2nd

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 2 of model 2 finished [ -6.97809224 -17.54141483   9.19703481  -3.85403336  -0.95230366
   3.4445167   -6.04295371  -6.0316752  -24.57203246   7.6830043
  18.91641126]
Accuracy is 0.4609375


fold 3 of model 2 finished [ -1.78874103 -18.29623374   8.19363829  -8.24758955  -1.27149833
   4.5462346   -5.4259728   -6.37847724 -25.99372524   7.50748019
  17.4694098 ]
Accuracy is 0.5703125


fold 4 of model 2 finished [ -1.33408539 -19.01272237  10.34989436  -4.95814061  -0.68417388
   5.21097666  -5.24453688  -5.78414024 -23.66380453   7.48203825
  14.87830496]
Accuracy is 0.52734375
 Average accuracy is  0.546875




pos 57
para [3e-06, 1500]
fold 0 of model 3 finished [-0.17861711  0.75688836  1.03870255  0.15709344  0.96935756  0.0249043
 -0.04398176  0.81295384  0.35647431  0.9539258  -0.09924169]
Accuracy is 0.5625


fold 1 of model 3 finished [-0.20363897  0.76277301  1.03054753  0.27433928  0.96654852  0.03490114
 -0.04614871  0.80850945  0.33855265  0.94792818 -0.10687863]
Acc

fold 2 of model 10 finished [ -9.03738262 -17.94058056   9.33523634  -7.33680242  -0.92860417
  19.58251098  -5.62692704  -5.41029379 -22.15093614   8.85100655
  34.86154557]
Accuracy is 0.62109375


fold 3 of model 10 finished [  1.92030488 -18.78676773   8.88002768 -14.5876807   -1.27095583
  20.32143203  -5.45110131  -5.79497714 -23.88296259   8.3421916
  29.92718514]
Accuracy is 0.54296875


fold 4 of model 10 finished [  2.17471809 -19.6704171   11.11797033 -11.12032711  -0.68763748
  20.69864946  -5.89081561  -5.39413447 -22.37898133   8.28142453
  26.5068655 ]
Accuracy is 0.61328125
 Average accuracy is  0.578125




pos 194
para [0.003, 3000]
fold 0 of model 11 finished [  -0.70876442 -286.04983958  135.1188655  -110.90956528  -19.47181405
   96.00512681  -25.90038104  -88.95382632 -330.32926807  135.2304436
  234.04874014]
Accuracy is 0.53515625


fold 1 of model 11 finished [ -36.85137458 -282.13045869  127.91813787  -83.92522469  -26.09801637
  106.07556304  -25.64612263  -9

fold 0 of model 18 finished [  64.20922898 -165.35604214   84.72168597 -142.46169051  -10.67236766
  353.83848034  -94.6867125   -39.57433713 -149.13339487   88.05019262
  342.5132266 ]
Accuracy is 0.53515625


fold 1 of model 18 finished [  20.36858324 -162.56111823   79.47703298  -79.4146925   -14.48549382
  377.53764243  -96.98073747  -45.18480052 -169.76054493   83.02901337
  359.49513793]
Accuracy is 0.53515625


fold 2 of model 18 finished [ -55.55320055 -153.8749257    71.38626173  -61.09377872  -14.91921653
  330.88527106  -80.63697776  -44.04479314 -160.13475681   72.19195468
  417.72950884]
Accuracy is 0.58984375


fold 3 of model 18 finished [  59.56051086 -161.86950022   69.24174422 -142.37387101  -17.77743945
  353.7365819   -95.10924991  -47.23900782 -175.6146648    69.29743707
  367.22456544]
Accuracy is 0.5390625


fold 4 of model 18 finished [  61.79092035 -171.11683321   88.71814838 -113.10304954  -12.75174402
  351.15941615  -96.27655923  -44.71388167 -167.35282938  

fold 4 of model 25 finished [-0.12840696  0.856615    1.00022159  0.47166744  0.9811877   0.07071126
 -0.03451372  0.84839698  0.48819119  0.92887325 -0.22442228]
Accuracy is 0.59765625
 Average accuracy is  0.54453125




pos 295
para [0.6000000000000001, 500]
fold 0 of model 26 finished [ -4620.44361209 -23896.14668339  11768.33093511 -12024.54392443
  -1778.53429994   7789.98172894  -5303.65561179  -7636.0584986
 -27846.56148768   9924.29202702  27097.88949882]
Accuracy is 0.66796875


fold 1 of model 26 finished [ -4943.96850027 -23329.22697743  11104.08469886  -5184.39290589
  -2295.65281847  14639.29662066 -10540.23722365  -7899.09405569
 -29009.83316628   9149.98057492  34306.89653777]
Accuracy is 0.65625


fold 2 of model 26 finished [-11884.23887618 -23343.91008578  10931.94000708  -6211.34371987
  -2208.81781725  11346.79049352  -3619.2339673   -8626.20636256
 -31283.39156776   7683.96586246  32962.68577204]
Accuracy is 0.63671875


fold 3 of model 26 finished [ -3185.8296679

fold 2 of model 33 finished [-0.19376397  0.86679831  0.99016184  0.49236107  0.98058291  0.02125794
 -0.03140093  0.8489358   0.49321985  0.92798865 -0.21421727]
Accuracy is 0.56640625


fold 3 of model 33 finished [-0.16317857  0.86476246  0.98958793  0.47639672  0.97935196  0.02554393
 -0.03262656  0.84722954  0.48600756  0.92438128 -0.22936915]
Accuracy is 0.58203125


fold 4 of model 33 finished [-0.13936448  0.86299105  0.99633856  0.47687921  0.98154535  0.03611577
 -0.03757927  0.84908879  0.49106722  0.92610038 -0.25114108]
Accuracy is 0.56640625
 Average accuracy is  0.55859375




pos 237
para [0.03, 1500]
fold 0 of model 34 finished [   76.65538609 -1520.56851297   722.9269299   -851.16330878
  -105.68335726   975.60827865  -270.40439272  -453.25038484
 -1679.14900005   716.84481952  1632.38899965]
Accuracy is 0.54296875


fold 1 of model 34 finished [ -215.71617152 -1499.17544237   688.42832681  -589.49874628
  -139.96834642  1062.74265912  -267.7111233   -507.601441
 -188

array([-0.09339412,  0.91345227,  0.97676829,  0.66901349,  0.98719773,
        0.01536375, -0.05107392,  0.87110375,  0.57077026,  0.92730486,
       -0.24136382])

In [11]:
fit()#3rd

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 3 of model 2 finished [-0.36512403 -0.59896946  1.65923213 -0.10864911  0.83108385  0.71792188
 -0.19555869  0.26478666 -1.60414211  1.45908046  1.22941806]
Accuracy is 0.62890625


fold 4 of model 2 finished [-0.54215884 -0.70689105  1.72323438 -0.50816496  0.84416692  0.6689137
 -0.17423895  0.24926428 -1.66144231  1.4133917   1.57521552]
Accuracy is 0.53515625
 Average accuracy is  0.56875




pos 260
para [0.08, 500]
fold 0 of model 3 finished [ -462.75037992 -1283.02412494   562.50814104  -981.2795153
  -150.54288314  2669.35499565  -711.84004287  -364.86103609
 -1252.12780765   571.49577506  3546.25256293]
Accuracy is 0.52734375


fold 1 of model 3 finished [ -355.82454185 -1591.93578842   740.71361703  -721.80182039
  -133.17942435  2759.58314284  -705.0594575   -415.21923347
 -1497.66781883   640.55566439  3147.88321906]
Accuracy is 0.6015625


fold 2 of model 3 finished [  210.9876697  -1439.30191786   664.44620242   -32.34647953
  -169.5654601   2894.01109994  -720.95832

fold 1 of model 10 finished [ -7.14980244 -21.53793254  11.12714797  -7.9296792   -0.89905017
  11.34632331 -11.24153129  -6.00940284 -24.55986306   9.05001763
  24.76987225]
Accuracy is 0.54296875


fold 2 of model 10 finished [ -1.34166843 -18.92125499   9.63873415  -1.69954656  -1.39844942
  13.09726757 -11.36933851  -6.33343141 -25.91597928   8.59799987
  17.82064481]
Accuracy is 0.53515625


fold 3 of model 10 finished [ -2.26869641 -17.1104869    9.35209624  -4.7002571   -0.81162837
  15.01495628  -4.09012472  -6.04034615 -24.10458631   8.37719609
  22.09679545]
Accuracy is 0.625


fold 4 of model 10 finished [ -5.80268444 -18.67045419  10.23368566  -9.85287706  -0.61162064
  11.89301815 -11.41291512  -6.42793792 -25.53516875   7.71884156
  24.72384022]
Accuracy is 0.5859375
 Average accuracy is  0.56328125




pos 118
para [6.000000000000001e-05, 2500]
fold 0 of model 11 finished [-1.84432831 -3.56038198  2.84373905 -1.8840366   0.46171988  1.89567348
 -0.51122017 -0.75265823 -5

fold 4 of model 17 finished [ -6.13384021 -28.3519532   15.11494358 -17.87054116  -1.40000828
  33.54635702  -8.48510762  -9.42787321 -36.07283164  11.8113749
  48.71601137]
Accuracy is 0.53125
 Average accuracy is  0.56484375




pos 190
para [0.003, 500]
fold 0 of model 18 finished [-11.96939418 -43.3121668   20.62036759 -31.77698856  -4.22514097
 108.56930575 -29.42190305 -11.07216617 -40.16655947  21.41876532
 132.69469236]
Accuracy is 0.52734375


fold 1 of model 18 finished [ -7.3868699  -54.26031785  26.91663545 -24.0553042   -3.65532635
 113.33958699 -28.56614853 -13.02849529 -49.50134909  23.84939439
 115.63824526]
Accuracy is 0.59765625


fold 2 of model 18 finished [  9.88933503 -48.7503866   24.07336263   0.74243112  -4.90411931
 116.96165147 -29.28487111 -14.55075333 -56.23463792  22.67589352
  92.00463886]
Accuracy is 0.55859375


fold 3 of model 18 finished [  4.54442022 -44.52587256  23.16291124 -11.56411254  -3.522072
 114.81090448 -30.45331257 -14.15288456 -52.2693646

fold 2 of model 25 finished [  -282.99040932  -7735.50371565   3280.48360061   -470.6016966
   -967.12490801   3170.52546259   -745.34684967  -2863.20071951
 -10547.49811292   2988.84283745   6103.74723138]
Accuracy is 0.5546875


fold 3 of model 25 finished [  -792.86924902  -7093.83503463   3175.66505548  -1234.42241103
   -719.9992684    3021.72285146   -793.06585016  -2806.11145789
 -10095.25923719   2848.136762     6668.89310293]
Accuracy is 0.62890625


fold 4 of model 25 finished [ -1568.92776187  -7771.4359662    3622.34722139  -2632.49103694
   -643.92605303   2736.3638266    -756.86928991  -2943.09363741
 -10607.16674385   2614.78038837   8164.02657115]
Accuracy is 0.55078125
 Average accuracy is  0.58125




pos 30
para [7e-07, 500]
fold 0 of model 26 finished [ 0.37098221  0.95383031  0.98481745  0.81957126  0.99298978  0.24360453
 -0.32553421  0.92661196  0.75654532  0.95622938  0.28661275]
Accuracy is 0.62890625


fold 1 of model 26 finished [ 0.35579322  0.94967169  0.98

fold 0 of model 33 finished [-1.67480226 -3.53217947  2.82575695 -1.65187859  0.46180134  1.55182001
 -0.41660764 -0.76820514 -5.25347864  2.43910417  5.15926202]
Accuracy is 0.52734375


fold 1 of model 33 finished [-1.36859917 -4.5919904   3.41887845 -1.06244141  0.51591889  1.65087503
 -0.39766913 -0.84839405 -5.75684565  2.81824604  4.65279498]
Accuracy is 0.59765625


fold 2 of model 33 finished [-0.34054219 -3.91572287  3.02379858 -0.05607737  0.38633459  1.93239302
 -0.4647871  -0.91060857 -5.99575643  2.71297698  3.45978817]
Accuracy is 0.55859375


fold 3 of model 33 finished [-0.61764856 -3.47345788  2.9516589  -0.60393738  0.53333723  1.83967312
 -0.48776045 -0.87517906 -5.70158788  2.64080776  3.77015261]
Accuracy is 0.62890625


fold 4 of model 33 finished [-1.05712574 -3.85729257  3.18164521 -1.56793874  0.58694412  1.72925225
 -0.42965234 -0.94220943 -5.96514589  2.48581669  4.63966718]
Accuracy is 0.53515625
 Average accuracy is  0.56953125




pos 1
para [1e-07, 1000]


array([ 0.66595014,  0.97558976,  0.99109864,  0.90095579,  0.99622998,
        0.43621695, -0.52714682,  0.959863  ,  0.86659551,  0.97542287,
        0.60266659])

In [18]:
fit()#4th

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 2 of model 2 finished [-8.41467116e+03 -3.27722438e+04  1.24292805e+04  4.88501488e+00
 -3.26594497e+03  5.08175029e+03 -2.09507225e+03 -1.10732784e+04
 -4.13434497e+04  1.39376197e+04  2.68822390e+04]
Accuracy is 0.6328125


fold 3 of model 2 finished [ -9707.58897358 -30262.21963859  11990.30728721   1671.07766648
  -2531.55170443   4630.06161577    393.77366854 -10591.83558725
 -37362.18728966  12604.74668041  25708.29705386]
Accuracy is 0.56640625


fold 4 of model 2 finished [ -9582.45263283 -31165.65240949  13722.08743483   4592.35673094
  -2450.5423754    1907.18351183  -7855.92698152 -11405.00644216
 -42798.07158257  13373.60692795  24592.1129258 ]
Accuracy is 0.4921875
 Average accuracy is  0.60390625




pos 105
para [4e-05, 500]
fold 0 of model 3 finished [-0.23595735  0.35045186  1.2149862   0.78573666  0.95274381  1.53647849
 -0.41142733  0.68839261 -0.08143087  1.26170623  1.08244221]
Accuracy is 0.5390625


fold 1 of model 3 finished [-0.01649082  0.28964371  1.2594

fold 0 of model 10 finished [-10.23206366 -48.65599463  21.2029577    6.6402611   -2.10112345
  22.18634315 -20.0749517  -18.46363695 -69.74241866  27.69959025
  46.83633958]
Accuracy is 0.609375


fold 1 of model 10 finished [ -0.12113943 -52.11422048  24.11559472  -8.43443612  -2.71097099
  26.33835592  -6.69882661 -17.67350222 -68.47876465  33.33626943
  45.19578525]
Accuracy is 0.6171875


fold 2 of model 10 finished [-19.40927588 -55.84667473  22.20953004  -4.32575574  -4.74796488
  22.55660847 -19.54645621 -17.8657331  -69.20251677  25.72763335
  52.86024864]
Accuracy is 0.578125


fold 3 of model 10 finished [-20.3741986  -52.30794128  21.1358613   -0.91106793  -3.6623933
  24.92987302  -6.43189489 -15.98269739 -58.17183008  25.64593017
  56.23733542]
Accuracy is 0.5859375


fold 4 of model 10 finished [-13.0915679  -53.7243186   23.83374431  12.62362575  -3.3376265
  27.42458412  -7.3120125  -17.08914363 -67.29515663  27.95953331
  52.17447002]
Accuracy is 0.5703125
 Average ac

fold 3 of model 17 finished [-0.3044347   0.51015178  1.11837332  0.17168421  0.94562782  0.09136831
 -0.11432261  0.72105105  0.04019132  1.02647538  0.14756725]
Accuracy is 0.5078125


fold 4 of model 17 finished [-0.27499651  0.49333378  1.14444829  0.24885988  0.94665809  0.10582937
 -0.12234484  0.711169   -0.03021319  1.03027733  0.14193561]
Accuracy is 0.58203125
 Average accuracy is  0.5546875




pos 133
para [9e-05, 2500]
fold 0 of model 18 finished [-1.46354681 -5.50568255  3.5915302   0.91395477  0.58788934  2.81311689
 -2.57290522 -1.62625918 -8.53522307  4.32718865  5.79138181]
Accuracy is 0.6015625


fold 1 of model 18 finished [-0.46906654 -5.98566629  3.95294209 -0.99635668  0.49659218  2.78971056
 -2.55173755 -1.57372524 -8.51201691  5.04237956  5.19936262]
Accuracy is 0.55859375


fold 2 of model 18 finished [-2.59240119 -6.41025936  3.70717114 -0.32429934  0.24889462  2.86649211
 -2.5200454  -1.56040026 -8.49191548  4.05745811  6.55491732]
Accuracy is 0.578125


fol

fold 2 of model 25 finished [-0.24394792  0.75560543  1.02537433  0.29471689  0.96552139  0.03194993
 -0.03133517  0.80830784  0.33907724  0.94809306 -0.08154826]
Accuracy is 0.59375


fold 3 of model 25 finished [-0.24349018  0.77028924  1.02012933  0.27860226  0.96988845  0.02990241
 -0.03326682  0.81426351  0.37214753  0.95189156 -0.0813523 ]
Accuracy is 0.5859375


fold 4 of model 25 finished [-0.21903335  0.76209132  1.03264297  0.32037237  0.97002554  0.04389823
 -0.02953413  0.8104984   0.34408749  0.9494861  -0.08631998]
Accuracy is 0.55859375
 Average accuracy is  0.58984375




pos 52
para [2e-06, 1500]
fold 0 of model 26 finished [-0.21377467  0.83206704  1.00336173  0.43766817  0.97905949  0.02970572
 -0.03425251  0.83211185  0.4314      0.945036   -0.15433758]
Accuracy is 0.6015625


fold 1 of model 26 finished [-0.18356932  0.82105429  1.01158582  0.43422852  0.97765305  0.03294323
 -0.03387604  0.82988047  0.4212192   0.94980239 -0.18514706]
Accuracy is 0.6015625


fold 

fold 1 of model 33 finished [-0.18780158  0.72456423  1.05415654  0.38916081  0.97110299  0.16539605
 -0.05328481  0.79765604  0.30303044  0.99935079 -0.03873099]
Accuracy is 0.6015625


fold 2 of model 33 finished [-0.29579631  0.71541121  1.04056865  0.38698309  0.96404999  0.1308578
 -0.14698322  0.79917089  0.30991862  0.97771052  0.00692273]
Accuracy is 0.5234375


fold 3 of model 33 finished [-0.30591557  0.73447401  1.03357307  0.38925233  0.96875613  0.12531813
 -0.14838171  0.80656374  0.35158071  0.9779939   0.01148254]
Accuracy is 0.484375


fold 4 of model 33 finished [-0.2536441   0.72548923  1.04917063  0.4576415   0.96964037  0.17718222
 -0.05880439  0.80298042  0.31915901  0.9811751   0.01624922]
Accuracy is 0.56640625
 Average accuracy is  0.54296875




pos 198
para [0.004, 2500]
fold 0 of model 34 finished [ -45.03159812 -282.22020856  117.42238438   40.91437574  -16.50955283
  152.62171756  -39.76121653 -108.30381006 -397.15370948  155.56254898
  285.16662829]
Accur

array([ -34002.28902092, -512497.34455691,  146117.04454449,
         18759.57984583,  -43382.62211988,   26691.56530999,
         -2468.22302594, -166312.13774585, -575573.15515018,
        303453.61955795,  244708.04079373])

In [21]:
fit()#5th

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 3 of model 2 finished [-0.12467916  0.9105682   0.97509208  0.67314611  0.98674627  0.0201699
 -0.04266408  0.8676131   0.56145748  0.92567727 -0.26459333]
Accuracy is 0.578125


fold 4 of model 2 finished [-0.13349639  0.91172111  0.97332667  0.67899849  0.98735668  0.02728892
 -0.04388063  0.86740432  0.55985677  0.92638748 -0.26483716]
Accuracy is 0.55078125
 Average accuracy is  0.5796875




pos 256
para [0.07, 1000]
fold 0 of model 3 finished [ -433.69198303 -2738.5503744   1177.37650971  -582.73089613
  -254.3537191   2568.88445385  -660.62965745  -827.74178076
 -2993.05853454  1082.09987321  3385.2415012 ]
Accuracy is 0.6171875


fold 1 of model 3 finished [-1049.47632778 -2580.77626453  1035.21821927   211.4321805
  -220.33501042  2629.22612023  -631.31848124  -801.15285686
 -2951.03295865  1146.71838854  3643.62838582]
Accuracy is 0.5703125


fold 2 of model 3 finished [ -111.21493995 -2342.5107103    953.38218696  -891.09296474
  -215.45364775  2417.14201978  -626.36212

fold 2 of model 10 finished [-3285.74567958 -7603.04755158  3175.02240045 -1375.58081947
  -698.1513727   3733.2682135  -2065.56922121 -2726.22247742
 -9394.5172476   3558.51840225 10745.72846542]
Accuracy is 0.6015625


fold 3 of model 10 finished [ -199.24143143 -8415.98673819  3398.64593786 -2823.92749924
  -865.4698238   6195.72076431 -1544.77091929 -2805.67539707
 -9924.40878802  3170.24849529  9305.56048601]
Accuracy is 0.55859375


fold 4 of model 10 finished [ -3104.51751023  -8176.21035148   3385.69944272   1682.41234295
   -637.21044068   5577.30428249  -4798.08948563  -3079.82881992
 -10787.11764571   3248.97531187   8897.62712491]
Accuracy is 0.53515625
 Average accuracy is  0.56875




pos 154
para [0.0004, 3000]
fold 0 of model 11 finished [ -7.48056715 -40.87529077  18.23310843  -6.99867117  -3.00681835
  12.38428734 -11.32087929 -12.64717539 -48.63253631  16.53950912
  32.45549711]
Accuracy is 0.5546875


fold 1 of model 11 finished [-12.5473846  -38.7255628   16.430235

fold 1 of model 18 finished [-0.45700699  0.41256089  1.16848563  0.62190925  0.94319349  1.01591544
 -0.89871085  0.71845038  0.02436046  1.14530777  0.65848479]
Accuracy is 0.515625


fold 2 of model 18 finished [-0.21597215  0.44877977  1.15567223  0.34229773  0.94521776  0.92166797
 -0.86778712  0.72459979  0.06858701  1.17600028  0.58324011]
Accuracy is 0.53125


fold 3 of model 18 finished [-0.18630045  0.40538056  1.172752    0.3371375   0.9343285   0.92959365
 -0.86735996  0.71297347  0.02185453  1.11619801  0.56563769]
Accuracy is 0.54296875


fold 4 of model 18 finished [-0.36650948  0.42407633  1.16865812  0.69123468  0.94820325  1.03269789
 -0.88471787  0.69787244 -0.02553327  1.12237074  0.54895148]
Accuracy is 0.515625
 Average accuracy is  0.52421875




pos 138
para [0.0001, 2500]
fold 0 of model 19 finished [-1.87896969 -7.95754851  4.67903419 -1.48008299  0.14170399  3.14783916
 -2.85793402 -1.99927253 -9.85465091  4.22666341  7.11405297]
Accuracy is 0.55078125


fold

fold 0 of model 26 finished [-0.25087392  0.63710777  1.0829579   0.21564589  0.95776429  0.10626518
 -0.11954224  0.77316001  0.21717662  0.97662298  0.02254839]
Accuracy is 0.53515625


fold 1 of model 26 finished [-0.27577987  0.65415778  1.07333594  0.28009431  0.96034371  0.13520869
 -0.03947283  0.77555048  0.22407891  0.98829908  0.05317599]
Accuracy is 0.57421875


fold 2 of model 26 finished [-0.2050936   0.67049775  1.06532251  0.18043726  0.96126886  0.11716203
 -0.04169698  0.77674342  0.24093968  0.99947424  0.0289337 ]
Accuracy is 0.55078125


fold 3 of model 26 finished [-2.13642401e-01  6.49952033e-01  1.07018823e+00  2.25641962e-01
  9.55652640e-01  8.92780427e-02 -1.15063048e-01  7.70498680e-01
  2.20303035e-01  9.71430924e-01 -7.33010309e-04]
Accuracy is 0.55078125


fold 4 of model 26 finished [-0.25058184  0.66071497  1.06726302  0.28622706  0.96337304  0.10921167
 -0.12001232  0.76614327  0.19981931  0.97722269  0.00884587]
Accuracy is 0.51953125
 Average accuracy

fold 0 of model 34 finished [-12997.17562839 -73254.38250998  31656.69683225  -1962.54050671
  -6626.13919731  10793.62206718     88.09660103 -24100.38768124
 -87275.71629037  23472.97494405  50232.38921896]
Accuracy is 0.5625


fold 1 of model 34 finished [-20102.46593113 -68221.15142677  28600.72838975    181.21435134
  -6133.73189789   2212.70931737 -10984.86895105 -24480.10110186
 -89391.41669858  25463.65750622  48579.20317084]
Accuracy is 0.46484375


fold 2 of model 34 finished [ -8028.16422972 -64802.36225271  26517.42228574  -7065.72047939
  -5927.76507262  12659.04080972  -5563.8173252  -23964.94639471
 -82978.44102144  29245.31592366  49310.86593804]
Accuracy is 0.59765625


fold 3 of model 34 finished [ -9952.17910099 -68380.02252286  26702.87148483  -8024.35438561
  -7205.66861089   5717.7559692  -20036.05471479 -25259.2492282
 -87993.57444132  20851.4450805   47362.44882742]
Accuracy is 0.484375


fold 4 of model 34 finished [-13001.51770885 -65682.98082679  26759.4252046

array([ 0.32473586,  0.94954179,  0.98384768,  0.8136251 ,  0.99267502,
        0.2287393 , -0.30144185,  0.92080798,  0.73689702,  0.95428197,
        0.22436967])

In [41]:
from sklearn.linear_model import LogisticRegression
X = X_all
y = y_all
clf = LogisticRegression(C=1500).fit(X, y)
r = clf.predict(X_test)
accuracy(r,y_test)
#clf.score(r, y_test)

/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7554858934169278

In [30]:
start = datetime.datetime.now()
fit()
end = datetime.datetime.now()
print (end-start)

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in double_scalars


fold 0 of model 0 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.4375


fold 1 of model 0 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.46875


fold 2 of model 0 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.46875


fold 3 of model 0 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.484375


fold 4 of model 0 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.47265625
 Average accuracy is  0.46640625




pos 76
para [7e-06, 1000]
fold 0 of model 1 finished [-0.2291159   0.69576613  1.06482999  0.29827433  0.96749834  0.14476704
 -0.17434091  0.78175412  0.25219451  0.99952359 -0.00321172]
Accuracy is 0.4921875


fold 1 of model 1 finished [-0.30439521  0.71282308  1.04692818  0.19554594  0.96789416  0.15938102
 -0.18239612  0.79221922  0.29160611  1.00459508  0.09335028]
Accuracy is 0.53515625


fold 2 of model 1 finished [-0.18986406  0.69501038  1.04766549  0.24085762  0.96134103 

In [3]:

fit()
end = datetime.datetime.now()
print (end-start)

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

fold 3 of model 1 finished [-0.17234107  0.88072898  0.98614747  0.57895995  0.98344605  0.03396832
 -0.03653167  0.85425757  0.51086766  0.92943826 -0.24904006]
Accuracy is 0.52734375


fold 4 of model 1 finished [-0.16875424  0.8830331   0.98305164  0.54573615  0.98366835  0.0247632
 -0.03357614  0.85359611  0.50936835  0.92997236 -0.24390693]
Accuracy is 0.53125
 Average accuracy is  0.55390625




pos 243
para [0.04, 2500]
fold 0 of model 2 finished [ -530.422255   -3088.40070843  1361.00023749  -237.27659075
  -245.66460253  1418.52529447  -329.83658053 -1153.31435461
 -4209.71423816  1609.73986503  2843.3148885 ]
Accuracy is 0.59375


fold 1 of model 2 finished [ -924.19226386 -2904.08613384  1160.5023995   -950.93973669
  -231.8191098   1335.05490786  -347.169848   -1065.64086151
 -3976.56352692  1670.91839898  3446.76871313]
Accuracy is 0.5625


fold 2 of model 2 finished [ -237.43533664 -3201.26401002  1166.22543534  -801.39668452
  -336.20760025  1474.07871166  -370.51521337 

/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in add
/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in double_scalars


fold 0 of model 5 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.4375


fold 1 of model 5 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.46875


fold 2 of model 5 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.46875


fold 3 of model 5 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.484375


fold 4 of model 5 finished [nan nan nan nan nan nan nan nan nan nan nan]
Accuracy is 0.47265625
 Average accuracy is  0.46640625




pos 317
para [1.0, 1500]
fold 0 of model 6 finished [-5.07246350e+05 -1.36090637e+07  2.14645242e+06  1.04130163e+04
 -1.38222064e+06 -3.65935876e+04  3.60055188e+04 -4.36024742e+06
 -1.26950407e+07  1.08657382e+07  4.90078829e+06]
Accuracy is 0.734375


fold 1 of model 6 finished [-5.00221298e+05 -1.08039192e+07 -3.56902717e+05 -2.26464005e+05
 -1.45236593e+06  2.62457398e+03  5.02965118e+04 -3.31596187e+06
 -9.96741489e+06  9.52927392e+06  4.10433670e+06]
Accuracy is 0.62890625




fold 2 of model 14 finished [ -141.04452963 -2404.98456358   914.35284465  -930.28489456
  -242.60368269  2508.95185383  -650.3213593   -718.30947582
 -2628.22572044  1246.66444262  2982.27336713]
Accuracy is 0.56640625


fold 3 of model 14 finished [-1192.47093259 -2282.94873137   992.23994298   -62.59008947
  -196.96175913  1595.66848351  -706.87411513  -731.41318735
 -2619.87525242  1078.32094255  3639.3145874 ]
Accuracy is 0.58203125


fold 4 of model 14 finished [ -846.94134424 -2245.39646848   927.03157381  -481.01227175
  -176.31977768  1658.01889832  -226.96266033  -800.34137186
 -2839.82162452   996.74376189  3664.20382295]
Accuracy is 0.52734375
 Average accuracy is  0.575


(array([0]),) 0 [4e-07, 3000]
best para are 4e-07 3000
(319,)
(319,)
The final accuracy is 0.5736677115987461
The confusion matrix of prediction is
 [[122  86]
 [ 50  61]]
0:07:36.137757


In [19]:
    bestalpha=8e-05
    bestn = 500
    w_final = findw(bestalpha, bestn, X_train[3], y_train[3])
    finalresult = predict(X_valid[3],w_final)
    finalaccu = accuracy(finalresult,y_valid[3])
    finalco = conmat(finalresult,y_valid[3])
    print(finalaccu)
    print(w_final)
    print(finalco)

(256,)
(256,)
0.5234375
[ 0.54933391 -0.41421511  1.63731827 -0.07773312  0.87077072  2.4864558
 -2.36371658  0.44271833 -1.00246637  1.61278958  1.69131922]
[[ 34  11]
 [111 100]]


In [43]:
    bestalpha=3.0000000000000004e-05
    bestn = 500
    w_final = findw(bestalpha, bestn, X_train[0], y_train[0])
    finalresult = predict(X_train[0],w_final)
    finalaccu = accuracy(finalresult,y_train[0])
    finalco = conmat(finalresult,y_train[0])
    print(finalaccu)
    print(w_final)
    print(finalco)

(1024,)
(1024,)
0.53515625
[-0.55268735  0.46666683  1.13202264  0.19808881  0.94172743  0.83216279
 -0.86827085  0.7357792   0.10855653  1.15100999  0.96520447]
[[122  45]
 [431 426]]


In [40]:
    bestalpha=3.0000000000000004e-05
    bestn = 500
    w_final = findw(bestalpha, bestn, X_all, y_all)
    finalresult = predict(X_all,w_final)
    finalaccu = accuracy(finalresult,y_all)
    finalco = conmat(finalresult,y_all)
    print(finalaccu)
    print(finalco)

(1280,)
(1280,)
0.5734375
[[667 528]
 [ 18  67]]


In [34]:

    wfinal=findw(3.0000000000000004e-05, 500, X_train[0],y_train[0])
    print(wfinal)
    a = predict(X_valid[0],wfinal)
    accu=accuracy(a,X_valid[0])
    co = conmat(a,y_valid[0])
    print(accu)
    print(co)
 

[-0.55268735  0.46666683  1.13202264  0.19808881  0.94172743  0.83216279
 -0.86827085  0.7357792   0.10855653  1.15100999  0.96520447]


TypeError: accuracy() takes 2 positional arguments but 3 were given

In [60]:
    wfinal=findw(0.001,1000,X_train[0],y_train[0])
    a = predict(X_valid[0],wfinal)
    accu=accuracy(a,y_valid[0])
    co = conmat(a,y_valid[0])
    print(accu)
    print(co)

(256,)
(256,)
0.57421875
[[55 15]
 [94 92]]


In [40]:
for i in range(9):#from 0.0000001-0.000001 1e-7~1e-6
    start = datetime.datetime.now()

    alpha = (i+1)*1e-7
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.658307210031348
[[151  80]
 [ 29  59]]
0:00:10.248174


(319,)
(319,)
0.6363636363636364
[[150  86]
 [ 30  53]]
0:00:10.007627


(319,)
(319,)
0.6269592476489029
[[149  88]
 [ 31  51]]
0:00:09.933093


(319,)
(319,)
0.5862068965517241
[[135  87]
 [ 45  52]]
0:00:10.390303


(319,)
(319,)
0.5893416927899686
[[135  86]
 [ 45  53]]
0:00:10.259773


(319,)
(319,)
0.5893416927899686
[[136  87]
 [ 44  52]]
0:00:10.612270


(319,)
(319,)
0.5924764890282131
[[137  87]
 [ 43  52]]
0:00:10.595049


(319,)
(319,)
0.6081504702194357
[[140  85]
 [ 40  54]]
0:00:10.409897


(319,)
(319,)
0.6050156739811913
[[140  86]
 [ 40  53]]
0:00:10.327679




In [42]:
for i in range(9):#from 0.000001-0.00001 1e-6~1e-5
    start = datetime.datetime.now()

    alpha = (i+1)*1e-6
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.6081504702194357
[[142  87]
 [ 38  52]]
0:00:10.484393


(319,)
(319,)
0.6175548589341693
[[143  85]
 [ 37  54]]
0:00:10.129912


(319,)
(319,)
0.5141065830721003
[[ 58  33]
 [122 106]]
0:00:10.266009


(319,)
(319,)
0.5015673981191222
[[ 45  24]
 [135 115]]
0:00:09.127075


(319,)
(319,)
0.47962382445141066
[[ 37  23]
 [143 116]]
0:00:09.927673


(319,)
(319,)
0.5862068965517241
[[179 131]
 [  1   8]]
0:00:10.710343


(319,)
(319,)
0.4827586206896552
[[ 33  18]
 [147 121]]
0:00:09.502064


(319,)
(319,)
0.5862068965517241
[[178 130]
 [  2   9]]
0:00:09.019073


(319,)
(319,)
0.4952978056426332
[[ 33  14]
 [147 125]]
0:00:09.041890




In [43]:
for i in range(9):#from 0.00001-0.0001 1e-5~1e-4
    start = datetime.datetime.now()

    alpha = (i+1)*1e-5
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.5862068965517241
[[178 130]
 [  2   9]]
0:00:09.025358


(319,)
(319,)
0.5454545454545454
[[ 60  25]
 [120 114]]
0:00:09.241525


(319,)
(319,)
0.6426332288401254
[[172 106]
 [  8  33]]
0:00:09.090701


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.376117


(319,)
(319,)
0.554858934169279
[[ 63  25]
 [117 114]]
0:00:09.143204


(319,)
(319,)
0.5454545454545454
[[ 59  24]
 [121 115]]
0:00:09.169750


(319,)
(319,)
0.47648902821316613
[[ 22   9]
 [158 130]]
0:00:09.257652


(319,)
(319,)
0.5329153605015674
[[ 55  24]
 [125 115]]
0:00:09.143028


(319,)
(319,)
0.5329153605015674
[[ 55  24]
 [125 115]]
0:00:09.601186




In [44]:
for i in range(9):#from 0.0001-0.001 1e-4~1e-3
    start = datetime.datetime.now()

    alpha = (i+1)*1e-4
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.47648902821316613
[[ 22   9]
 [158 130]]
0:00:09.019624


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:09.329799


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:09.230416


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:09.190479


(319,)
(319,)
0.47648902821316613
[[ 22   9]
 [158 130]]
0:00:09.069907


(319,)
(319,)
0.6238244514106583
[[174 114]
 [  6  25]]
0:00:09.165375


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:11.352492


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:10.023016


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:09.961681




In [45]:
for i in range(9):#from 0.001-0.01 1e-3~1e-2
    start = datetime.datetime.now()

    alpha = (i+1)*1e-3
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:09.339449


(319,)
(319,)
0.5799373040752351
[[179 133]
 [  1   6]]
0:00:09.321613


(319,)
(319,)
0.6300940438871473
[[174 112]
 [  6  27]]
0:00:09.461709


(319,)
(319,)
0.6300940438871473
[[174 112]
 [  6  27]]
0:00:09.472402


(319,)
(319,)
0.6300940438871473
[[174 112]
 [  6  27]]
0:00:10.098849


(319,)
(319,)
0.5768025078369906
[[180 135]
 [  0   4]]
0:00:10.678498


(319,)
(319,)
0.6300940438871473
[[174 112]
 [  6  27]]
0:00:09.829534


(319,)
(319,)
0.64576802507837
[[172 105]
 [  8  34]]
0:00:09.590914


(319,)
(319,)
0.64576802507837
[[172 105]
 [  8  34]]
0:00:09.534347




In [46]:
for i in range(9):#from 0.01-0.1
    start = datetime.datetime.now()

    alpha = (i+1)*1e-2
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.64576802507837
[[172 105]
 [  8  34]]
0:00:09.582727


(319,)
(319,)
0.6520376175548589
[[172 103]
 [  8  36]]
0:00:09.645034


(319,)
(319,)
0.6489028213166145
[[172 104]
 [  8  35]]
0:00:09.466325


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.613950


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.671792


(319,)
(319,)
0.4608150470219436
[[ 10   2]
 [170 137]]
0:00:09.490789


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.453882


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.691835


(319,)
(319,)
0.5768025078369906
[[179 134]
 [  1   5]]
0:00:10.216013




In [47]:
for i in range(9):#from 0.1-1
    start = datetime.datetime.now()

    alpha = (i+1)*1e-1
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.6865203761755486
[[161  81]
 [ 19  58]]
0:00:10.540491


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:10.466971


(319,)
(319,)
0.6833855799373041
[[166  87]
 [ 14  52]]
0:00:09.590352


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.456771


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.380877


(319,)
(319,)
0.5047021943573667
[[ 27   5]
 [153 134]]
0:00:09.567612


(319,)
(319,)
0.6927899686520376
[[110  28]
 [ 70 111]]
0:00:09.440878


(319,)
(319,)
0.5642633228840125
[[180 139]
 [  0   0]]
0:00:09.446812


(319,)
(319,)
0.6520376175548589
[[ 83  14]
 [ 97 125]]
0:00:11.030434




In [48]:
for i in range(9):#from 1-10
    start = datetime.datetime.now()

    alpha = (i+1)*1e-0
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

(319,)
(319,)
0.64576802507837
[[ 87  20]
 [ 93 119]]
0:00:10.513181


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.559556




/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in add
/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in double_scalars


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.599654


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.509688


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.489268


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.455401


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.508011


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.527046


(319,)
(319,)
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:10.242022




In [49]:
for i in range(9):#from 10-100
    start = datetime.datetime.now()

    alpha = (i+1)*10
    wfinal=findw(alpha,2000,data2)
    a = predict(testset,wfinal)
    accu=accuracy(a,testset)
    co = conmat(a,testset)
    print('used alhpa is ',alpha)
    print(accu)
    print(co)
    
    end = datetime.datetime.now()
    print (end-start)
    print('\n')

/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in double_scalars
/home/mz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in add


(319,)
(319,)
used alhpa is  10
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:10.936107


(319,)
(319,)
used alhpa is  20
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.417473


(319,)
(319,)
used alhpa is  30
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.189862


(319,)
(319,)
used alhpa is  40
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.252055


(319,)
(319,)
used alhpa is  50
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.230196


(319,)
(319,)
used alhpa is  60
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.247373


(319,)
(319,)
used alhpa is  70
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.350979


(319,)
(319,)
used alhpa is  80
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.197263


(319,)
(319,)
used alhpa is  90
0.43573667711598746
[[  0   0]
 [180 139]]
0:00:09.304717




In [25]:
from sklearn.linear_model import LogisticRegression
X = data2[:,0:data2.shape[1]-1]
y = data2[:,[data2.shape[1]-1]]
clf = LogisticRegression().fit(X, y)
r = clf.predict(testset[:,0:data2.shape[1]-1])
accuracy(r,testset)
label=testset[:,data2.shape[1]-1]
print(label)
clf.score(testset[:,0:data2.shape[1]-1], label)

[0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1.
 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0.
 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1.
 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 0. 0.]


/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7586206896551724

In [20]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X = trainset[1][:,[2,9,10]]
y = trainset[1][:,[data2.shape[1]-1]]
clf = LogisticRegression().fit(X, y)
r = clf.predict(testset[:,[2,9,10]])
accuracy(r,testset)
label=testset[:,data2.shape[1]-1]
print(label)
clf.score(testset[:,[2,9,10]], label)

[1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1.
 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0.
 1. 1. 1. 1. 0. 1. 1.]


/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7053291536050157

In [1]:
wfinal=findw(0.01,2500,data2)
a = predict(testset,wfinal)
accu=accuracy(a,testset)
print(accu)

NameError: name 'findw' is not defined

In [16]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
a = [1,2,7,8,9,10]
X = trainset[1][:,a]
y = trainset[1][:,[data2.shape[1]-1]]
clf = LogisticRegression().fit(X, y)
r = clf.predict(testset[:,a])
accuracy(r,testset)
label=testset[:,data2.shape[1]-1]
print(label)
clf.score(testset[:,a], label)

[0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 0. 1. 1.]


/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/mz/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6614420062695925

In [66]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
a = [0]
X = trainset[1][:,a]
y = trainset[1][:,[data2.shape[1]-1]]
clf = LogisticRegression().fit(X, y)
r = clf.predict(testset[:,a])
accuracy(r,testset)
label=testset[:,data2.shape[1]-1]
print(label)
clf.score(testset[:,a], label)

SyntaxError: invalid syntax (<ipython-input-66-900ae5aaad1e>, line 3)

In [64]:
rr=data2[0:10,0:4]
print(rr)
np.random.shuffle(rr)
print(rr)

[[ 6.4    0.31   0.09   1.4  ]
 [ 8.3    0.66   0.15   1.9  ]
 [ 8.     0.25   0.43   1.7  ]
 [ 6.9    0.41   0.33   2.2  ]
 [ 7.     0.685  0.     1.9  ]
 [ 5.3    0.47   0.11   2.2  ]
 [ 7.2    0.605  0.02   1.9  ]
 [ 8.7    0.675  0.1    1.6  ]
 [10.     0.32   0.59   2.2  ]
 [ 6.6    0.61   0.01   1.9  ]]
[[ 6.6    0.61   0.01   1.9  ]
 [ 8.7    0.675  0.1    1.6  ]
 [ 8.3    0.66   0.15   1.9  ]
 [ 6.4    0.31   0.09   1.4  ]
 [ 7.     0.685  0.     1.9  ]
 [ 6.9    0.41   0.33   2.2  ]
 [ 5.3    0.47   0.11   2.2  ]
 [ 7.2    0.605  0.02   1.9  ]
 [10.     0.32   0.59   2.2  ]
 [ 8.     0.25   0.43   1.7  ]]


In [35]:
rv = testset[:,-1]
print(rv)

[0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 0. 0. 1. 0. 1. 1.]


In [28]:
a = [1,0,1,0,1,1,1]
b = [0,1,1,0,1,0,0]
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [56]:
metrix = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])


print(metrix)

[[1 2 3 4]
 [5 6 7 8]]


In [41]:
print(1e-0)

1.0


In [4]:
fit()

[[1e-07, 500], [1e-07, 1000], [1e-07, 1500], [1e-07, 2500], [1e-07, 3000], [2e-07, 500], [2e-07, 1000], [2e-07, 1500], [2e-07, 2500], [2e-07, 3000], [3e-07, 500], [3e-07, 1000], [3e-07, 1500], [3e-07, 2500], [3e-07, 3000], [4e-07, 500], [4e-07, 1000], [4e-07, 1500], [4e-07, 2500], [4e-07, 3000], [5e-07, 500], [5e-07, 1000], [5e-07, 1500], [5e-07, 2500], [5e-07, 3000], [6e-07, 500], [6e-07, 1000], [6e-07, 1500], [6e-07, 2500], [6e-07, 3000], [7e-07, 500], [7e-07, 1000], [7e-07, 1500], [7e-07, 2500], [7e-07, 3000], [8e-07, 500], [8e-07, 1000], [8e-07, 1500], [8e-07, 2500], [8e-07, 3000], [9e-07, 500], [9e-07, 1000], [9e-07, 1500], [9e-07, 2500], [9e-07, 3000], [1e-06, 500], [1e-06, 1000], [1e-06, 1500], [1e-06, 2500], [1e-06, 3000], [2e-06, 500], [2e-06, 1000], [2e-06, 1500], [2e-06, 2500], [2e-06, 3000], [3e-06, 500], [3e-06, 1000], [3e-06, 1500], [3e-06, 2500], [3e-06, 3000], [4e-06, 500], [4e-06, 1000], [4e-06, 1500], [4e-06, 2500], [4e-06, 3000], [4.9999999999999996e-06, 500], [4.999

array([ -17911.63817889, -119717.63372219,   52038.15766594,
         -6824.03931897,  -10806.47595623,    4310.383205  ,
        -20656.0012517 ,  -47234.18382459, -171996.51930633,
         50628.33455867,   77603.07190444])